In [1]:
# import boto3
# import pandas as pd
# import io
# import re
# import yaml
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import shap
# from scipy.stats import pearsonr
# from scipy import stats
# # import plotly.express as px
# import pickle
# import random
# import pandas as pd
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
# from os.path import join
# import warnings
# warnings.filterwarnings('ignore')
# from collections import Counter
# from sklearn.metrics import r2_score
# from sklearn.impute import SimpleImputer
# from sklearn.ensemble import RandomForestRegressor
# import os.path
# import pandas as pd
# import io
# import re
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import shap
# from scipy.stats import pearsonr
# import pickle
# import random
# import pandas as pd
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
# from os.path import join
# import warnings
# from scipy import stats
# import yaml
# from sklearn.impute import SimpleImputer
# import boto3
# import csv
# from PIL import Image
# from io import BytesIO
# import cv2

In [1]:
import torch
from super_gradients.training import models


ModuleNotFoundError: No module named 'torch'

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_ARCH = 'yolo_nas_s'  # choose 'yolo_nas_m' or 'yolo_nas_s' for medium or small model


In [6]:
yolo_nas_s = models.get('yolo_nas_s', pretrained_weights="coco").to(DEVICE)
yolo_nas_m = models.get('yolo_nas_m', pretrained_weights="coco").to(DEVICE)
yolo_nas_l = models.get('yolo_nas_l', pretrained_weights="coco").to(DEVICE)


In [9]:
from pympler import asizeof

print(f"The size of the yolo_nas_s is {asizeof.asizeof(yolo_nas_s)/1024**2} GB")
print(f"The size of the yolo_nas_m is {asizeof.asizeof(yolo_nas_m)/1024**2} GB")
print(f"The size of the yolo_nas_l is {asizeof.asizeof(yolo_nas_l)/1024**2} GB")

In [45]:
asizeof.asizeof(yolo_nas_s)/1024**2

3.6126861572265625

In [44]:
asizeof.asizeof(yolo_nas_m)/1024**2

3.782958984375

In [43]:
asizeof.asizeof(yolo_nas_l)/1024**2

4.293006896972656

In [4]:
model = models.get(MODEL_ARCH, pretrained_weights="coco").to(DEVICE)

Downloading: "https://sghub.deci.ai/models/yolo_nas_s_coco.pth" to C:\Users\itays/.cache\torch\hub\checkpoints\yolo_nas_s_coco.pth
100%|█████████████████████████████████████████████████████████████████████████████| 73.1M/73.1M [01:15<00:00, 1.01MB/s]


In [12]:
model.get_parameter

<bound method Module.get_parameter of YoloNAS_S(
  (backbone): NStageBackbone(
    (stem): YoloNASStem(
      (conv): QARepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (branch_3x3): Sequential(
          (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        )
        (branch_1x1): Conv2d(3, 48, kernel_size=(1, 1), stride=(2, 2))
        (post_bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (rbr_reparam): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
    )
    (stage1): YoloNASStage(
      (downsample): QARepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (branch_3x3): Sequential(
          (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(9

In [14]:
yolo_nas_s

YoloNAS_S(
  (backbone): NStageBackbone(
    (stem): YoloNASStem(
      (conv): QARepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (branch_3x3): Sequential(
          (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        )
        (branch_1x1): Conv2d(3, 48, kernel_size=(1, 1), stride=(2, 2))
        (post_bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (rbr_reparam): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
    )
    (stage1): YoloNASStage(
      (downsample): QARepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (branch_3x3): Sequential(
          (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=Tr

In [21]:
IMAGES_PATH = fr'C:\Users\itays\Desktop\accumulated_files\images'
images_names = os.listdir(IMAGES_PATH)

In [25]:
image = cv2.imread(join(IMAGES_PATH, images_names[0]))

In [29]:
result = list(model.predict(image, conf=0.35))[0]

In [32]:
import supervision as sv

detections = sv.Detections(
    xyxy=result.prediction.bboxes_xyxy,
    confidence=result.prediction.confidence,
    class_id=result.prediction.labels.astype(int)
)

box_annotator = sv.BoxAnnotator()

labels = [
    f"{result.class_names[class_id]} {confidence:0.2f}"
    for _, _, confidence, class_id, _
    in detections
]

annotated_frame = box_annotator.annotate(
    scene=image.copy(),
    detections=detections,
    labels=labels
)

%matplotlib inline
sv.plot_image(annotated_frame, (12, 12))

ModuleNotFoundError: No module named 'supervision'